# Import Libraries

In [1]:
import pandas as pd
import numpy as np
#!pip install mlxtend
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import warnings
warnings.filterwarnings('ignore')

# Read Data

In [2]:
df = pd.read_csv("SALESDATASET.csv")
df.head()



,order_id,status_so,status_fo_leading,order_time,code_item,item_descr,quantity,selling_price,total_amount
0,43155,FULFILLED,PACKED,4/7/22 0:02,1,Toothbrush Soft,1.0,5.61,5.61
1,43155,FULFILLED,PACKED,4/7/22 0:02,2,Day Care Toothbrush,1.0,3.14,3.14
2,43156,FULFILLED,PACKED,4/7/22 0:05,3,Denture Container,1.0,0.00,0.00
3,43156,FULFILLED,PACKED,4/7/22 0:05,4,Denture Cleaner,4.0,45.90,183.60
4,43157,FULFILLED,PACKED,4/7/22 0:17,5,Fruit Pestels,2.0,10.08,20.16


# Overview of Data

In [3]:
df.shape

(3868, 9)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3868 entries, 0 to 3867
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   order_id           3868 non-null   int64  
 1   status_so          3868 non-null   object 
 2   status_fo_leading  3866 non-null   object 
 3   order_time         3868 non-null   object 
 4   code_item          3868 non-null   int64  
 5   item_descr         3868 non-null   object 
 6   quantity           3844 non-null   float64
 7   selling_price      3868 non-null   float64
 8   total_amount       3868 non-null   float64
dtypes: float64(3), int64(2), object(4)
memory usage: 272.1+ KB


In [5]:
df.describe()

,order_id,code_item,quantity,selling_price,total_amount
count,3868.000000,3868.000000,3844.000000,3868.000000,3868.000000
mean,44335.936143,38.548087,3.275234,28.748214,127.447479
std,702.061860,27.404894,3.616569,39.752989,306.473322
min,42959.000000,1.000000,1.000000,0.000000,0.000000
25%,43752.750000,15.000000,1.000000,4.587500,6.202500
50%,44295.500000,31.000000,2.000000,17.000000,37.160000
75%,44926.000000,63.000000,4.000000,37.050000,164.160000
max,45611.000000,109.000000,48.000000,627.520000,8232.000000


# Data Pre-Processing

## Handling missing values

In [6]:
#find null columns with null values
df.isnull().sum()

order_id              0
status_so             0
status_fo_leading     2
order_time            0
code_item             0
item_descr            0
quantity             24
selling_price         0
total_amount          0
dtype: int64

In [7]:
#get rows with missing value in quantity
missing_qty_rows = df[df['quantity'].isnull()==True].index

In [8]:
#get quantity by using total amount/ selling price
for i in missing_qty_rows:
    df['quantity'][i] = df['total_amount'][i]/df['selling_price'][i]

In [9]:
#check if quantity filled
df.iloc[missing_qty_rows]

,order_id,status_so,status_fo_leading,order_time,code_item,item_descr,quantity,selling_price,total_amount
732,43612,CANCEL,CANCEL,7/7/22 1:13,9,Vitamin Water,24.0,26.67,640.08
733,43612,CANCEL,CANCEL,7/7/22 1:13,7,Stationeries,NaN,0.00,0.00
1097,43839,CANCEL,CANCEL,7/7/22 11:00,73,Vitamin Gummies (Multi Flavour) 200g,NaN,0.00,0.00
1098,43839,CANCEL,CANCEL,7/7/22 11:00,51,Vitamin Gummies (Blackberry Flavour) 100g,2.0,27.00,54.00
1099,43839,CANCEL,CANCEL,7/7/22 11:00,50,Vitamin Gummies (Orange Flavour) 100g,NaN,0.00,0.00
1103,43838,CANCEL,CANCEL,7/7/22 11:01,73,Vitamin Gummies (Multi Flavour) 200g,NaN,0.00,0.00
1104,43838,CANCEL,CANCEL,7/7/22 11:01,50,Vitamin Gummies (Orange Flavour) 100g,NaN,0.00,0.00
1105,43838,CANCEL,CANCEL,7/7/22 11:01,51,Vitamin Gummies (Blackberry Flavour) 100g,2.0,27.00,54.00
1316,43947,FULFILLED,PACKED,7/7/22 14:38,13,Kids Teeth ToothPaste 55ml,1.0,6.78,6.78
1382,43994,CLOSE,CANCEL,7/7/22 15:59,71,Fresh Mint Toothpaste,1.0,13.60,13.60


In [10]:
#remaining empty quantities
df[df['quantity'].isnull()==True]

,order_id,status_so,status_fo_leading,order_time,code_item,item_descr,quantity,selling_price,total_amount
733,43612,CANCEL,CANCEL,7/7/22 1:13,7,Stationeries,NaN,0.0,0.0
1097,43839,CANCEL,CANCEL,7/7/22 11:00,73,Vitamin Gummies (Multi Flavour) 200g,NaN,0.0,0.0
1099,43839,CANCEL,CANCEL,7/7/22 11:00,50,Vitamin Gummies (Orange Flavour) 100g,NaN,0.0,0.0
1103,43838,CANCEL,CANCEL,7/7/22 11:01,73,Vitamin Gummies (Multi Flavour) 200g,NaN,0.0,0.0
1104,43838,CANCEL,CANCEL,7/7/22 11:01,50,Vitamin Gummies (Orange Flavour) 100g,NaN,0.0,0.0
1383,43994,CLOSE,CANCEL,7/7/22 15:59,72,Travel Pack Toothpaste,NaN,0.0,0.0
2070,44376,CANCEL,CANCEL,9/7/22 12:51,41,Backpack,NaN,0.0,0.0


In [11]:
#drop rows where quantity is nan
df = df.dropna(subset= ['quantity'])

In [12]:
#get rows with missing values in status fo leading column
missing_status_rows = df[df['status_fo_leading'].isnull()==True].index

In [13]:
#replace na values with 'CANCEL'
for i in missing_qty_rows:
    df['status_fo_leading'][i] = 'CANCEL'

In [14]:
#check for any other missing values
df.isnull().sum()

order_id             0
status_so            0
status_fo_leading    0
order_time           0
code_item            0
item_descr           0
quantity             0
selling_price        0
total_amount         0
dtype: int64

## Data Types

In [15]:
#check data types for each column
df.dtypes

order_id               int64
status_so             object
status_fo_leading     object
order_time            object
code_item              int64
item_descr            object
quantity             float64
selling_price        float64
total_amount         float64
dtype: object

In [16]:
#convert order_id to string
df.order_id = df.order_id.astype(str)

#convert order-time to datetime
df.order_time = pd.to_datetime(df.order_time)

#convert quantity to interger
df.quantity = df.quantity.astype(int)

df.dtypes

order_id                     object
status_so                    object
status_fo_leading            object
order_time           datetime64[ns]
code_item                     int64
item_descr                   object
quantity                      int64
selling_price               float64
total_amount                float64
dtype: object

## Remove freebies (Stationaries, Pen, Skillet and Backpack)

In [17]:
remove = ['Pen', 'Stationeries', 'Backpack', 'Skillet']

for item in remove:
    df = df[df['item_descr'] != item]

# Format table for Apriori Analysis

In [18]:
#reformat df for apriori analysis
#group by order id and item description
#sum quantity
#pivot column labels
#reset index
#fill nan values w 0
#set orderid as index
basket = df.groupby(['order_id','item_descr'])['quantity'].sum().unstack().reset_index().fillna(0).set_index('order_id')

In [19]:
basket.head()

item_descr,14 Day Whitening Toothpaste,Abrasion Cream,All Round Toothpaste,Boroca,Boroca (50),Calcium Flavoured Tablet (60),Calcium Tablet (100),Calcium Tablet (200),Calcium Tablet (30),Calcium Tablet (500),...,Vitamin Gummies (Strawberry Flavour) 200g,Vitamin Gummies (Watermelon Flavour) 100g,Vitamin Gummies (Watermelon Flavour) 50g,Vitamin Tablets 100mg,Vitamin Water,Vitamin Water Orange,Whitening Toothpaste,XTRA Fresh Toothpaste,XTRA Mint Toothpaste,XTRA Whitening Toothpaste
order_id,,,,,,,,,,,,,,,,,,,,,
42959,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42977,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
43005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
#function to change quantities to occurences 
#as the association analysis considers itemsets 
def encode_values(x):
    if x <= 0:
        return 0
    elif x >= 1:
        return 1

In [21]:
#apply encode values function to every element of basket 
basket_encoded = basket.applymap(encode_values)
basket_encoded.head()

item_descr,14 Day Whitening Toothpaste,Abrasion Cream,All Round Toothpaste,Boroca,Boroca (50),Calcium Flavoured Tablet (60),Calcium Tablet (100),Calcium Tablet (200),Calcium Tablet (30),Calcium Tablet (500),...,Vitamin Gummies (Strawberry Flavour) 200g,Vitamin Gummies (Watermelon Flavour) 100g,Vitamin Gummies (Watermelon Flavour) 50g,Vitamin Tablets 100mg,Vitamin Water,Vitamin Water Orange,Whitening Toothpaste,XTRA Fresh Toothpaste,XTRA Mint Toothpaste,XTRA Whitening Toothpaste
order_id,,,,,,,,,,,,,,,,,,,,,
42959,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
42977,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
43005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43012,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43013,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Top 5 Items

In [22]:
top_df = df.groupby('item_descr').sum()
top_df = top_df.sort_values(by = 'quantity', ascending = False)
top_df.head()

,code_item,quantity,selling_price,total_amount
item_descr,,,,
Vitamin Water Orange,2595,857,4515.17,27261.01
Rash Cream,522,778,2803.03,33672.26
Repair Toothpaste,1056,607,2428.27,19932.35
Retainer Cleaner,3600,584,3357.18,18380.04
Toothbrush 3 pack,2520,506,2384.11,18652.31


# Training Model

In [23]:
frequent_itemsets = apriori(basket_encoded, min_support= 0.01, use_colnames=True)

In [24]:
rules = association_rules(frequent_itemsets, metric='lift', min_threshold=1)

In [25]:
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Fresh Mint Toothpaste),(Travel Pack Toothpaste),0.071275,0.073851,0.067411,0.945783,12.806563,0.062147,17.082296
1,(Travel Pack Toothpaste),(Fresh Mint Toothpaste),0.073851,0.071275,0.067411,0.912791,12.806563,0.062147,10.649377
2,(Vitamin Gummies (Multi Flavour) 100g),(Vitamin Gummies (Blackberry Flavour) 100g),0.038214,0.043796,0.016745,0.438202,10.005618,0.015072,1.702044
3,(Vitamin Gummies (Blackberry Flavour) 100g),(Vitamin Gummies (Multi Flavour) 100g),0.043796,0.038214,0.016745,0.382353,10.005618,0.015072,1.557178
4,(Vitamin Gummies (Multi Flavour) 200g),(Vitamin Gummies (Blackberry Flavour) 100g),0.045942,0.043796,0.016316,0.355140,8.109034,0.014304,1.482810


In [26]:
#create new columns to show number of items in antecedent and consequent
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules["consequents_len"] = rules["consequents"].apply(lambda x: len(x))

# RESULTS

## Single Antecedent

In [27]:
#setting filters and thresholds
single = rules[ (rules['antecedent_len'] == 1) &
               (rules['consequents_len'] == 1) &
               (rules['confidence'] > 0.5) &
               (rules['lift'] > 1) ]

In [28]:
print("There are {} Relation derived.".format(len(single)))


There are 8 Relation derived.


In [29]:
#sort table to show results with highest lift first
single = single.sort_values(by='lift',ascending=False)
single = single.reset_index(drop = True)


In [30]:
#function to print out all rules and their metrics
def show_rules(dataframe):
    for row in dataframe.index:
        #print itemset
        print("Rule: " + str(dataframe['antecedents'][row])[12:-3] + " -> " + str(dataframe['consequents'][row])[12:-3])

        #print support
        print("Support: " + str(dataframe['support'][row]))

        #print confidence
        print("Confidence: " + str(dataframe['confidence'][row]))

        #print lift
        print("Lift: " + str(dataframe['lift'][row]))
        print("=====================================")

show_rules(single)




Rule: Vitamin Gummies (Blackberry Flavour) 100g -> Vitamin Gummies (Orange Flavour) 100g
Support: 0.0347788750536711
Confidence: 0.7941176470588235
Lift: 21.017045454545453
Rule: Vitamin Gummies (Orange Flavour) 100g -> Vitamin Gummies (Blackberry Flavour) 100g
Support: 0.0347788750536711
Confidence: 0.9204545454545454
Lift: 21.017045454545453
Rule: Vitamin Gummies (Strawberry Flavour) 100g -> Vitamin Gummies (Orange Flavour) 100g
Support: 0.0210390725633319
Confidence: 0.6363636363636362
Lift: 16.841942148760328
Rule: Vitamin Gummies (Orange Flavour) 100g -> Vitamin Gummies (Strawberry Flavour) 100g
Support: 0.0210390725633319
Confidence: 0.5568181818181818
Lift: 16.841942148760328
Rule: Vitamin Gummies (Strawberry Flavour) 100g -> Vitamin Gummies (Blackberry Flavour) 100g
Support: 0.0210390725633319
Confidence: 0.6363636363636362
Lift: 14.530303030303028
Rule: Vitamin Gummies (Strawberry Flavour) 100g -> Vitamin Gummies (Multi Flavour) 100g
Support: 0.016745384285100903
Confidence: 0

## Double Antecedent

In [31]:
#setting filters and thresholds
double = rules[ (rules['antecedent_len'] == 2) &
               (rules['consequents_len'] == 1) &
               (rules['confidence'] > 0.75) &
               (rules['lift'] > 1) ]

In [32]:
print("There are {} Relation derived.".format(len(double)))

There are 12 Relation derived.


In [33]:
#sort table to show results with highest lift first
double = double.sort_values(by='lift',ascending=False)
double = double.reset_index(drop = True)

In [34]:
show_rules(double)

Rule: Vitamin Gummies (Multi Flavour) 100g', 'Vitamin Gummies (Blackberry Flavour) 100g -> Vitamin Gummies (Strawberry Flavour) 100g
Support: 0.016745384285100903
Confidence: 1.0
Lift: 30.246753246753244
Rule: Vitamin Gummies (Multi Flavour) 100g', 'Vitamin Gummies (Orange Flavour) 100g -> Vitamin Gummies (Strawberry Flavour) 100g
Support: 0.016316015457277802
Confidence: 0.9500000000000001
Lift: 28.734415584415583
Rule: Vitamin Gummies (Multi Flavour) 100g', 'Vitamin Gummies (Blackberry Flavour) 100g -> Vitamin Gummies (Orange Flavour) 100g
Support: 0.016316015457277802
Confidence: 0.9743589743589743
Lift: 25.787296037296038
Rule: Vitamin Gummies (Strawberry Flavour) 100g', 'Vitamin Gummies (Multi Flavour) 100g -> Vitamin Gummies (Orange Flavour) 100g
Support: 0.016316015457277802
Confidence: 0.9743589743589743
Lift: 25.787296037296038
Rule: Vitamin Gummies (Strawberry Flavour) 100g', 'Vitamin Gummies (Blackberry Flavour) 100g -> Vitamin Gummies (Orange Flavour) 100g
Support: 0.020180

# Recommendations

The results of the Apriori Analysis have uncovered several insights regarding association between products. With these results, our recommendations are as follows:

Firstly, Vitamin Gummies of size 100g of any flavour can be bundled together in 2s and 3s at a slightly discounted rate to increase sales. Alternatively, the company can target users who purchase 1 bottle of vitamin gummies and upsell them additional bottles before check out under the "people who buy this also buy this" section. 

Our reasoning for this due to the strong likelihood that Vitamin Gummies are purchased together. This is evident from the Lift metric for all results with Vitamin Gummies. As the Lift metric for all these results are greater than 1, it indicates that the second item is likely to be bought if the first item is bought. Therefore, we would strongly recommend this strategy to increase the basket size and sales revenue. 

Secondly, we would recommend bundling Fresh Mint Toothpaste and Travel Pack Toothpaste together.

Our reasoning for this is also due to the strong relationship between these 2 products. From the results, we can see that the Lift metric for these 2 items is 12 which is much greater than 1, indicating that item 2 is very likely to be bought if item 1 is bought. Hence, we would recommend bundling these 2 items together at an attractive price to potentially increase basket size. 